In [1]:
import os

import numpy as np
from scipy.spatial.distance import cosine

import pandas as pd

import pyspark

from hog_utils import extract_features

conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','8g')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/15 15:51:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/opt/conda/envs/bigdata-fall22/lib/python3.7/site-packages/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [2]:
# get the list of images
img_list = sc.parallelize([file.path for file in os.scandir('images/')])

# calculate their HOG features
hogs = img_list.map(extract_features)

# zip names along to recognize the results
hogs = hogs.zip(img_list)

In [3]:
# select the image for testing
test_img_name = "road0.png"
test_hog = sc.parallelize([extract_features(test_img_name)])

In [4]:
# get a pair of (test_img, img) for each img in the corpus
prod_vec = hogs.cartesian(test_hog)

# compute cosine similarity on each pair while preserving the img names
sims = prod_vec.map(lambda x: (cosine(x[0][0], x[1]), x[0][1]))

In [5]:
# take top 5 sorted descending by similarity 
top5 = sims.takeOrdered(5, key=lambda x: -x[0])
top5

22/12/15 15:52:02 WARN TaskSetManager: Stage 0 contains a task of very large size (2061 KiB). The maximum recommended task size is 1000 KiB.


[(0.9270216098937593, 'images/road47.png'),
 (0.9092798967338858, 'images/road155.png'),
 (0.9020340883294123, 'images/road529.png'),
 (0.901370899288721, 'images/road81.png'),
 (0.899622166973055, 'images/road39.png')]